In [ ]:
#all headers
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.preprocessing import MinMaxScaler # to convert or normalize all values to values bw 0 and 1

In [ ]:
#mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd  /content/drive/MyDrive/License plate detection project/train20X20

/content/drive/MyDrive/License plate detection project/train20X20


In [ ]:

#preparation of dataset
import cv2
import os
X_train=[]
Y_train=[]
def read_resize_images(folder_path, target_shape=(28, 28)):
    images = []
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
        if img is not None:
            img_resized = cv2.resize(img, target_shape)  # Resize image
            img_flattened = img_resized.flatten()  # Flatten image into 1D array
            images.append(img_flattened)
    return np.array(images)

for i in range(10):
  folder_path = str(i)
  target_shape = (28, 28)  # Target shape for resizing

  # Read and resize images
  images_data = read_resize_images(folder_path, target_shape)
  X_train.extend(images_data)
  for j in range(len(images_data)):
     Y_train.append(i)
# Convert lists to NumPy arrays

for i in range(26):

  folder_path = chr(ord('A') + i)
  target_shape = (28, 28)  # Target shape for resizing

  # Read and resize images
  images_data = read_resize_images(folder_path, target_shape)
  X_train.extend(images_data)
  for j in range(len(images_data)):
     Y_train.append(10+i)
X_train = np.array(X_train)
Y_train = np.array(Y_train)


# Display shape of the array
print("Shape of the array containing resized images:", X_train,X_train.shape,len(Y_train),Y_train)





Shape of the array containing resized images: [[255 255 255 ... 255 254 253]
 [251 253 255 ... 255 254 254]
 [255 253 252 ... 255 254 254]
 ...
 [149 144 139 ... 100 100 100]
 [158 153 145 ... 156 214 255]
 [251 253 249 ...  76 137 220]] (384, 784) 384 [ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  2  2  2
  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4
  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6
  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8
  8  9  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11 12 12 12 12 12 12
 12 12 12 12 13 13 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14 14 14
 14 14 14 15 15 15 15 15 15 15 15 15 15 16 16 16 16 16 16 16 16 16 16 17
 17 17 17 17 17 17 17 17 17 18 18 18 18 18 18 18 19 19 19 19 19 19 19 19
 19 19 19 19 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 21 21 21 21 21
 21 21 21 21 21 2

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train) #convert or normalize all values of input values bw 0 and 1
#X_test = scaler.fit_transform(X_test)
X_train=X_train.T
Y_train=Y_train.T
print(X_train)

[[1.         0.98431373 1.         ... 0.58431373 0.61960784 0.98431373]
 [1.         0.99212598 0.99212598 ... 0.56299213 0.5984252  0.99212598]
 [1.         1.         0.98818898 ... 0.54330709 0.56692913 0.97637795]
 ...
 [1.         1.         1.         ... 0.38492063 0.60714286 0.28968254]
 [0.996      0.996      0.996      ... 0.38       0.836      0.528     ]
 [0.99193548 0.99596774 0.99596774 ... 0.375      1.         0.85887097]]


In [ ]:
def create_params():
    """Initialise the weights and biases using Kaiming He initialisation for ReLU
    and Xavier for softmax
    """
    # Kaiming He Initialisation
    he_std = sqrt(2) / (784)
    # Xavier Initialisation
    lower, upper = -(1 / sqrt(36)), (1 / sqrt(36))

    W1 = np.random.randn(36, 784) * he_std
    b1 = np.random.randn(36, 1)
    W2 = lower + np.random.rand(36, 36) * (upper - lower)
    b2 = np.random.randn(36, 1)

    return W1, b1, W2, b2

In [ ]:
def ReLU(X):
    """ReLU function"""
    return np.maximum(0,X)

def softmax(X):
    """Softmax function"""
    e_X = np.exp(X - np.max(X))
    return e_X / e_X.sum(axis=0)

def deriv_ReLU(X):
    """Derivative of ReLU function"""
    return X > 0

def one_hot_encoder(y):
    """One Hot Encoder"""
    oh_y = np.zeros((y.size, y.max() + 1))
    oh_y[np.arange(y.size), y] = 1
    return oh_y.T

In [ ]:
def forward_prop(W1, b1, W2, b2, X):
    """Forward propagation steps"""
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def back_prop(Z1, A1, Z2, A2, W2, X, y):
    """Backwards propagation steps"""
    m = y.size
    oh_y = one_hot_encoder(y)
    dZ2 = A2 - oh_y
    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T)
    db1 = (1 / m) * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(pred, y):
    #print(pred, y)
    return np.sum(pred == y) / y.size

def grad_descent(X, y, iterations, alpha):
    W1, b1, W2, b2 = create_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 20 == 0:
            print(f'Epoch: {i}')
            print(f'Accuracy: {get_accuracy(get_predictions(A2), y)}\n')
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = grad_descent(X_train, Y_train, 1000, 0.05)
#print("W1: ",W1,"W2: ",W2,"b1: ",b1,"b2: ",b2)


In [ ]:
def model_pred(W1, b1, W2, b2, X):
    """Forward propagation steps"""
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    pred = get_predictions(A2)
    return pred

In [ ]:
from google.colab.patches import cv2_imshow
folder_path="X_test"
for filename in os.listdir(folder_path):
      filepath = os.path.join(folder_path, filename)
      img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
      if img is not None:
          img_resized = cv2.resize(img, target_shape)  # Resize image
          img_flattened = char[i]  # Flatten image into 1D array
          img_flattened = img_flattened.reshape(-1, 1)
          test_pred = model_pred(W1, b1, W2, b2, img_flattened)
          cv2_imshow(img)
          print(test_pred,len(img_flattened))




#get_accuracy(test_pred, y_test)

[17] 784


[28] 784
